In [1]:
import numpy as np
import pandas as pd

## Linsentypen

- A Sammellinse
- B Sammellinse
- G Sammellinse
- E Streulinse

## Allgemeine Settings

In [3]:
pos_lampe = 0e-2
u_pos_lampe = 0.1e-2

pos_linse_a = 5e-2
u_pos_linse_a = 0.1e-2

pos_gitter = 20e-2
u_pos_gitter = 0.1e-2

## Autokollimationsmethode Einfachlinsen

In [4]:
d_linse_spiegel = 3e-2
u_d_linse_spiegel = 0.1e-2

## Besselmethode Einfachlinsen

In [ ]:
gittergroesse = 5e-3
u_gittergroesse = 0.1e-2

gittergroesse_schirm = 20e-3
u_gittergroesse_schirm = 0.1e-3

g_pos_schirm = 69.5e-2
u_g_pos_schirm = 0.1e-2

e_pos_schirm = 84.1e-2
u_e_pos_schirm = 0.1e-2


## Autokollimationsmethode Linsensysteme

In [3]:
d_linse_spiegel = 3e-2
u_d_linse_spiegel = 0.1e-2

## Abbeverfahren Linsensysteme